<a href="https://colab.research.google.com/github/talleshenry/Data_Science/blob/master/postgraduate/data_visualization/An%C3%A1lise_descritiva_Talles_Henrique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processamento

In [0]:
import altair as alt
import pandas as pd

In [0]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-24-2020.csv'
data_covid19 = pd.read_csv(url, error_bad_lines=False)

In [0]:
data_covid19.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-03-24 23:37:31,34.223334,-82.461707,1,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-03-24 23:37:31,30.295065,-92.414197,2,0,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-03-24 23:37:31,37.767072,-75.632346,1,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-03-24 23:37:31,43.452658,-116.241552,19,0,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-03-24 23:37:31,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [0]:
confirmed_by_country = data_covid19[['Country_Region', 'Confirmed']].groupby(['Country_Region']).sum()
reference_geographic_vy_country = data_covid19[['Country_Region', 'Lat', 'Long_']].groupby(['Country_Region']).mean()
confirmed_cases = pd.merge(confirmed_by_country, reference_geographic_vy_country, on='Country_Region')
confirmed_cases.head()

,Confirmed,Lat,Long_
Country_Region,,,
Afghanistan,74,33.93911,67.709953
Albania,123,41.15330,20.168300
Algeria,264,28.03390,1.659600
Andorra,164,42.50630,1.521800
Angola,3,-11.20270,17.873900


In [0]:
data_covid19['Last_Update'] = pd.to_datetime(data_covid19['Last_Update'])
data_covid19['Data'] = data_covid19['Last_Update'].dt.strftime('%Y-%m-%d')
data_covid19.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Data
0,45001.0,Abbeville,South Carolina,US,2020-03-24 23:37:31,34.223334,-82.461707,1,0,0,0,"Abbeville, South Carolina, US",2020-03-24
1,22001.0,Acadia,Louisiana,US,2020-03-24 23:37:31,30.295065,-92.414197,2,0,0,0,"Acadia, Louisiana, US",2020-03-24
2,51001.0,Accomack,Virginia,US,2020-03-24 23:37:31,37.767072,-75.632346,1,0,0,0,"Accomack, Virginia, US",2020-03-24
3,16001.0,Ada,Idaho,US,2020-03-24 23:37:31,43.452658,-116.241552,19,0,0,0,"Ada, Idaho, US",2020-03-24
4,19001.0,Adair,Iowa,US,2020-03-24 23:37:31,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US",2020-03-24


In [0]:
cases_per_day =  data_covid19[['Data', 'Confirmed']].groupby(['Data']).sum()
cases_per_day

,Confirmed
Data,
2020-02-23,19
2020-03-08,1066
2020-03-12,93
2020-03-14,1018
2020-03-16,251
2020-03-18,146
2020-03-19,75
2020-03-20,254
2020-03-23,4136


In [0]:
map = alt.topo_feature("https://cdn.jsdelivr.net/npm/world-atlas@2/countries-110m.json", "countries")

# Análise univariada

Vamos fazer análise do numero de casos confirmados de mortes por corona vírus por país. Para isso utilizaremos um gráfico de barras.


In [0]:


alt.Chart(data_covid19[data_covid19['Deaths'] > 100]).mark_bar().encode(
    x='Deaths',
    y='Country_Region',
    color='Country_Region'
)

alt.Chart(...)

In [0]:


background = alt.Chart(map).mark_geoshape(
    fill='#ccffff',
    stroke='white'
).properties(
    width=1000,
    height=700
).project('naturalEarth1')

points = alt.Chart(data_covid19).mark_circle().encode(
    longitude='Long_:Q',
    latitude='Lat:Q',
    size='Confirmed:Q',
    color=alt.value('red'),
    )

background + points


alt.LayerChart(...)

In [0]:
alt.Chart(cases_per_day.reset_index()).mark_line().encode(
    x='Data:T',
    y='Confirmed:Q'
).properties(
    width=800,
    height=300
)

alt.Chart(...)

# Análise multivariada

In [0]:
selecao = alt.selection_interval(encodings=['x'])
p1 = alt.Chart(data_covid19).mark_point().encode(
    alt.X('Last_Update:T', scale=alt.Scale(domain=selecao)),
    y = "Deaths:Q",
    tooltip = "Active:Q"
).properties(
    width=600,
    height=200
)

p2 = alt.Chart(data_covid19).mark_area().encode(
    x = "Last_Update:T",
    y = "Confirmed:Q"
).properties(
    width=600,
    height = 60
).add_selection(selecao)

p1 & p2

alt.VConcatChart(...)

In [0]:
confirmed_by_date = data_covid19[['Data', 'Confirmed', 'Deaths']].groupby(['Data']).sum()
reference_geographic_by_date = data_covid19[['Data', 'Lat', 'Long_']].groupby(['Data'], as_index=False).mean()
confirmed_cases = pd.merge(confirmed_by_date, reference_geographic_by_date, on='Data')
confirmed_cases.head()

,Data,Confirmed,Deaths,Lat,Long_
0,2020-02-23,19,0,33.71895,92.04400
1,2020-03-08,1066,9,36.46930,101.23325
2,2020-03-12,93,1,43.66610,126.19230
3,2020-03-14,1018,4,27.61040,111.70880
4,2020-03-16,251,2,31.12160,103.82625


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Data,date_number
0,45001.0,Abbeville,South Carolina,US,2020-03-24 23:37:31,34.223334,-82.461707,1,0,0,0,"Abbeville, South Carolina, US",2020-03-24,20200324
1,22001.0,Acadia,Louisiana,US,2020-03-24 23:37:31,30.295065,-92.414197,2,0,0,0,"Acadia, Louisiana, US",2020-03-24,20200324
2,51001.0,Accomack,Virginia,US,2020-03-24 23:37:31,37.767072,-75.632346,1,0,0,0,"Accomack, Virginia, US",2020-03-24,20200324
3,16001.0,Ada,Idaho,US,2020-03-24 23:37:31,43.452658,-116.241552,19,0,0,0,"Ada, Idaho, US",2020-03-24,20200324
4,19001.0,Adair,Iowa,US,2020-03-24 23:37:31,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US",2020-03-24,20200324


In [0]:



alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    tooltip='Name:N'
).add_selection(selecao).transform_filter(selecao)

In [0]:

source = data_covid19[data_covid19['Deaths'] > 100]
brush = alt.selection(type='interval', encodings=['x'])

bars = alt.Chart().mark_bar().encode(
    x='Country_Region:N',
    y='Recovered:Q',
    opacity=alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.7)),
).add_selection(
    brush
)

line = alt.Chart().mark_rule(color='firebrick').encode(
    y='Recovered:Q',
    size=alt.SizeValue(3)
).transform_filter(
    brush
)

alt.layer(bars, line, data=source)

alt.LayerChart(...)